<a href="https://colab.research.google.com/github/OscarChang46/de-identify-sensitive-data-for-cybersecurity-use-cases-e.g.-PII-and-PHI-data-in-unstructured-text-an/blob/main/LSTM_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [15]:
# sample data
def load_data():
    # passengers number of international airline , 1949-01 ~ 1960-12 per month
    seq_number = np.array(
        [112., 118., 132., 129., 121., 135., 148., 148., 136., 119., 104.,
         118., 115., 126., 141., 135., 125., 149., 170., 170., 158., 133.,
         114., 140., 145., 150., 178., 163., 172., 178., 199., 199., 184.,
         162., 146., 166., 171., 180., 193., 181., 183., 218., 230., 242.,
         209., 191., 172., 194., 196., 196., 236., 235., 229., 243., 264.,
         272., 237., 211., 180., 201., 204., 188., 235., 227., 234., 264.,
         302., 293., 259., 229., 203., 229., 242., 233., 267., 269., 270.,
         315., 364., 347., 312., 274., 237., 278., 284., 277., 317., 313.,
         318., 374., 413., 405., 355., 306., 271., 306., 315., 301., 356.,
         348., 355., 422., 465., 467., 404., 347., 305., 336., 340., 318.,
         362., 348., 363., 435., 491., 505., 404., 359., 310., 337., 360.,
         342., 406., 396., 420., 472., 548., 559., 463., 407., 362., 405.,
         417., 391., 419., 461., 472., 535., 622., 606., 508., 461., 390.,
         432.], dtype=np.float32)
    # assert seq_number.shape == (144, )
    # plt.plot(seq_number)
    # plt.ion()
    # plt.pause(1)
    seq_number = seq_number[:, np.newaxis]

    # print(repr(seq))
    # 1949~1960, 12 years, 12*12==144 month
    seq_year = np.arange(12)
    seq_month = np.arange(12)
    seq_year_month = np.transpose(
        [np.repeat(seq_year, len(seq_month)),
         np.tile(seq_month, len(seq_year))],
    )  # Cartesian Product

    seq = np.concatenate((seq_number, seq_year_month), axis=1)

    # normalization
    seq = (seq - seq.mean(axis=0)) / seq.std(axis=0)
    return seq

In [16]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(RNN, self).__init__()
        self.W_xh = nn.Linear(input_size, hidden_size)
        self.W_hh = nn.Linear(input_size, hidden_size)

    def __call__(self, x, hidden):
        return self.step(x, hidden)

    def step(self, x, hidden):
        h1 = self.W_hh(hidden)
        w1 = self.W_xh(x)
        out = nn.Tanh(h1 + w1)
        hidden = self.W_hh.weight
        return out, hidden

In [17]:
rnn = RNN(20, 50)
rnn

RNN(
  (W_xh): Linear(in_features=20, out_features=50, bias=True)
  (W_hh): Linear(in_features=20, out_features=50, bias=True)
)

## LSTM

In [19]:
from torch.autograd import Variable
import torch
class LSTM(nn.Module):
  def __init__(self, input_size, hidden_size, cell_size):
    super(LSTM, self).__init__()
    self.gate = nn.Linear(input_size + hidden_size, cell_size) 
    self.hidden_size = hidden_size
    self.cell_size = cell_size
    self.output = nn.Linear(hidden_size, input_size)
    self.sigmoid =nn.Sigmoid()
    self.tanh = nn.Tanh()
    self.softmax = nn.Softmax()


  def forward(self, input, hidden, cell):
    combined = torch.cat((input, hidden), 1)
    f_gate = self.gate(combined)
    f_gate = self.sigmoid(f_gate)
    i_gate = self.gate(combined)
    i_gate = self.sigmoid(i_gate)
    o_gate = self.gate(combined)
    o_gate = self.sigmoid(o_gate)
    c_tilde = self.gate(combined)
    c_tilde = self.tanh(c_tilde)
    cell = torch.add(torch.mul(cell, f_gate) + torch.mul(c_tilde, i_gate))
    hidden = torch.mul(self.tanh(cell, o_gate))
    output = self.output(hidden)
    output = self.softmax(output)

    return output, hidden, cell
  
  def initHidden(self):
    return Variable(torch.zeros(1, self.hidden_size))

  def initCell(self):
    return Variable(torch.zeros(self.cell_size))

input_dim = 10
hidden_dim = 5
cell_dim = 5
output_dim = 10
lstm = LSTM(input_dim, hidden_dim, cell_dim)
# lstm
seq_len = 108 # the length of sentence
batch_size = 20# the number of sentence in the same batch
input_dim = 3# dimension of vector representation
input_data = torch.randn(108, 20, 3)
lstm

LSTM(
  (gate): Linear(in_features=15, out_features=5, bias=True)
  (output): Linear(in_features=5, out_features=10, bias=True)
  (sigmoid): Sigmoid()
  (tanh): Tanh()
  (softmax): Softmax(dim=None)
)

In [ ]:
class BiLSTM(nn.Module):
    
    def __init__(self):
        super(BiLSTM, self).__init__()
        self.hidden_size = 64
        drp = 0.1
        self.embedding = nn.Embedding(max_features, embed_size)
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = False
        self.lstm = nn.LSTM(embed_size, self.hidden_size, bidirectional=True, batch_first=True)
        self.linear = nn.Linear(self.hidden_size*4 , 64)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(drp)
        self.out = nn.Linear(64, 1)


    def forward(self, x):
        h_embedding = self.embedding(x)
        h_embedding = torch.squeeze(torch.unsqueeze(h_embedding, 0))
        
        h_lstm, _ = self.lstm(h_embedding)
        avg_pool = torch.mean(h_lstm, 1)
        max_pool, _ = torch.max(h_lstm, 1)
        #print("avg_pool", avg_pool.size())
        #print("max_pool", max_pool.size())
        conc = torch.cat(( avg_pool, max_pool), 1)
        conc = self.relu(self.linear(conc))
        conc = self.dropout(conc)
        out = self.out(conc)
        return out
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


# Conditional Random Field
https://github.com/typoverflow/pytorch-crf

In [ ]:
import torch
import sys
from matplotlib.pyplot import polar
import os
from numpy.lib.npyio import load
from feature_function import StatusFF, ObsFF, TransFF
class CRF(nn.Module):
    def __init__(self, d:LabelDict, ffshape):
        super(CRF, self).__init__()
        self.d = d
        self.L = len(d)
        self.K = 1
        for d in ffshape:
            self.K *= d
        self.w = nn.Parameter(torch.zeros(self.K))